In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Rescaling                                                  # type: ignore
from tensorflow.keras.models import Sequential                                                                                   # type: ignore
from tensorflow.keras.callbacks import EarlyStopping                                                                             # type: ignore
import matplotlib.pyplot as plt                                                                                                  
import os
import sys
import keras
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("Python version:", sys.version)
print("CPU Count:", os.cpu_count())

In [ ]:
train_another_dir = r'X:\nasim_xhqpjmy\Code\MLops\hurricane-damage\dataset\train_another'
val_dir = r'X:\nasim_xhqpjmy\Code\MLops\hurricane-damage\dataset\validation_another'
test_dir = r'X:\nasim_xhqpjmy\Code\MLops\hurricane-damage\dataset\test'
test_another_dir = r'X:\nasim_xhqpjmy\Code\MLops\hurricane-damage\dataset\test_another'

In [ ]:
img_size = (128, 128)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_another_dir,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size
)

In [ ]:
#Augmentation 
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    # optional zoom simulation (light)
    scale = tf.random.uniform([], 0.9, 1.1)
    new_size = tf.cast(tf.convert_to_tensor(img_size, dtype=tf.float32) * scale, tf.int32)
    image = tf.image.resize(image, new_size)
    image = tf.image.resize_with_crop_or_pad(image, img_size[0], img_size[1])
    return image, label

In [ ]:
# Apply augmentation only to training set
AUTOTUNE = tf.data.AUTOTUNE
train_ds = (train_ds
            .map(augment, num_parallel_calls=AUTOTUNE)
            .cache()
            .shuffle(1000)
            .prefetch(buffer_size=AUTOTUNE))

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def build_cnn(input_shape=(128, 128, 3)):
    model = Sequential([
        # Normalize & augment 
        Rescaling(1./255, input_shape=input_shape),

        # First conv block
        Conv2D(8, 3, activation='relu', padding='same'),
        MaxPooling2D(),

        #Second conv block 
        Conv2D(16, 3, activation='relu', padding='same'),
        MaxPooling2D(),

        # Third conv block — depthwise-separable
        tf.keras.layers.SeparableConv2D(32, 3, activation='relu', padding='same'),
        tf.keras.layers.GlobalAveragePooling2D(),

        #dense head
        Dense(32, activation='relu'),
        Dropout(0.2),

        #Output
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [ ]:
def train_model(train_ds, val_ds, epochs=5):
    model = build_cnn()
    print(model.summary())

    es = EarlyStopping(patience=3, restore_best_weights=True)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[es],
        verbose=1
    )
    return model, history

In [ ]:
model, history = train_model(train_ds, val_ds, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Training Progress")
plt.show()

In [ ]:
model.save("hurricane.h5")